In [4]:
import tensorflow as tf

# Load Xception Model

In [6]:
# Create a new folder and unzip the file located a permanent location into into this new folder.
!tar -xvzf /content/drive/MyDrive/DS-Data/model-Malaria-Detection-xception.tar.gz -C model-Malaria-Detection-xception1

model-Malaria-Detection-xception/
model-Malaria-Detection-xception/variables/
model-Malaria-Detection-xception/variables/variables.index
model-Malaria-Detection-xception/variables/variables.data-00000-of-00001
model-Malaria-Detection-xception/assets/
model-Malaria-Detection-xception/keras_metadata.pb
model-Malaria-Detection-xception/saved_model.pb


In [5]:
# load the saved model.
loaded_model = tf.keras.models.load_model('model-Malaria-Detection-xception1/model-Malaria-Detection-xception')

# Convert Xception Model to H5 file format

In [8]:
#Save model to h5 format 
loaded_model.save("model-Malaria-Detection-xception2.h5", save_format="h5")

# Converting h5 file to tflite

In [ ]:
# # convert h5 to tflite
# import os
# def get_file_size(file_path):
#     size = os.path.getsize(file_path)
#     return size

# def convert_bytes(size, unit=None):
#     if unit == "KB":
#         return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
#     elif unit == "MB":
#         return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
#     else:
#         return print('File size: ' + str(size) + ' bytes')

# from keras.models import load_model
# # Load h5 Model
# loaded_model2 = load_model("model-Malaria-Detection-xception2.h5")

# TF_LITE_MODEL_FILE_NAME = "model_malaria_pred_tflite_model.tflite"
# tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model2)
# tflite_model = tf_lite_converter.convert()
# tflite_model_name = TF_LITE_MODEL_FILE_NAME
# open(tflite_model_name, "wb").write(tflite_model)
# convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

# # convert h5 model to tflite 
# converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model2)
# tflite_model = converter.convert()
# # or using another method

# # Save the tflite model.
# with open(TF_LITE_MODEL_FILE_NAME, 'wb') as f:
#   f.write(tflite_model)

In [9]:
! pip install -q kaggle

In [10]:
 ! mkdir ~/.kaggle

In [11]:
! cp kaggle.json ~/.kaggle/

In [12]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [13]:
!kaggle datasets download -d miracle9to9/files1 

100% 524M/525M [00:12<00:00, 43.4MB/s]
100% 525M/525M [00:12<00:00, 44.0MB/s]


In [ ]:
!unzip files1.zip

In [7]:
#Import Required Libraries

import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.applications.xception import Xception, preprocess_input
from keras.layers import Dense, MaxPool2D, Conv2D, Dropout, Flatten, \
    GlobalAveragePooling2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings("ignore")

In [25]:
#Data Augmentation from the Given Data
datagen = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             zca_whitening=False,
                             rotation_range=10,
                             zoom_range=0.1,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False,
                             rescale=1/255.0, 
                             validation_split=0.2)

In [26]:
trainDatagen = datagen.flow_from_directory(directory='/content/Malaria Cells/training_set',
                                           target_size=(71,71),
                                           class_mode = 'binary',
                                           batch_size = 22048,
                                           subset='training')

Found 22048 images belonging to 2 classes.


In [ ]:
# trainDatagen1=[]
# for trn in trainDatagen:
#   trainDatagen1.append(trn)

In [27]:
valDatagen = datagen.flow_from_directory(directory='/content/Malaria Cells/training_set',
                                           target_size=(71,71),
                                           class_mode = 'binary',
                                           batch_size = 5510,
                                           subset='validation')

Found 5510 images belonging to 2 classes.


In [22]:
valDatagen1=[]
# for val in valDatagen:
#   valDatagen1.append(val)

# Another Method of Model Compression

In [ ]:
# https://medium.com/@chengweizhang2012/how-to-compress-your-keras-model-x5-smaller-with-tensorflow-model-optimization-adcfa9c9fe94

!pip install tensorflow_model_optimization
!pip install tf-nightly

In [11]:
import tensorflow_model_optimization as tfmot
model_for_export = tfmot.sparsity.keras.strip_pruning(loaded_model)

In [ ]:
help(model_for_export.fit)

In [13]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = 22048 #train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(loaded_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 3, 3, 2048)        41406643  
                                                                 
 prune_low_magnitude_global_  (None, 2048)             1         
 average_pooling2d (PruneLow                                     
 Magnitude)                                                      
                                                                 
 prune_low_magnitude_dropout  (None, 2048)             1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense (  (None, 64)               262210    
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_dense_1  (None, 1)                1

In [41]:
trainDatagen

In [28]:
import tempfile
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(trainDatagen,
                  batch_size=batch_size, epochs=epochs, 
                  callbacks=callbacks)

Epoch 1/2


UnimplementedError: ignored

In [ ]:
!pip install keras_cv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 203 kB 28.1 MB/s 


In [ ]:
from keras_cv.layers import BaseImageAugmentationLayer

ModuleNotFoundError: ignored

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow_model_optimization.sparsity import keras as sparsity


# Backend agnostic way to save/restore models
# _, keras_file = tempfile.mkstemp('.h5')
# print('Saving model to: ', keras_file)
# tf.keras.models.save_model(model, keras_file, include_optimizer=False)

keras_file = 'model-Malaria-Detection-xception2.h5'
# Load the serialized model
loaded_model = tf.keras.models.load_model(keras_file)

num_train_samples = 22048 
batch_size = 32
epochs = 4
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)
new_pruned_model.summary()

new_pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

ModuleNotFoundError: ignored

In [ ]:
# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
logdir= 'logdir'
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

new_pruned_model.fit(trainDatagen,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=valDatagen)

score = new_pruned_model.evaluate(valDatagen, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/4
1378/1378 [==============================] - 3510s 3s/step - loss: 0.0000e+00 - accuracy: 0.5311 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/4
  36/1378 [..............................] - ETA: 54:09 - loss: 0.0000e+00 - accuracy: 0.5104